In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
#### Load "derecha diario" dataset from csv.

In [2]:
df = pd.read_csv('./data/derecha_econ_news.csv')

In [3]:
df.shape

(339, 7)

In [4]:
df.head(5)

,Unnamed: 0,date,flyer,title,lead,body,url
0,0,30/11/2020,Economía,Economía española: en 12 meses se perdieron 10...,"Debido a la segunda ola del coronavirus, se pr...",Según el registro de la Seguridad Social españ...,https://derechadiario.com.ar/economia/economia...
1,1,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
2,2,29/11/2020,Economía,El reconocido ejecutivo Mark Mobius recomienda...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
3,3,28/11/2020,Economía,Ajuste a los jubilados: el aumento por decreto...,El Gobierno decretó un pobre aumento en torno ...,Alberto Fernández consolidó su reforma del sis...,https://derechadiario.com.ar/economia/ajuste-a...
4,4,27/11/2020,Economía,Brasil creó 395.000 puestos de trabajo en octu...,El Ministerio de Economía de Brasil confirmó u...,Con el acelerado ritmo de actividad en la econ...,https://derechadiario.com.ar/economia/brasil-c...


### Topics
Find from title the different topics inside the section about economy.

In [5]:
exchange = df.title[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo')]
exchange.shape

(32,)

In [6]:
inflation = df.title[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec')]
inflation.shape

(38,)

### Corpus (Body)
Get the body of those news which cover topics about inflation and the rate of exchange

In [7]:
inflation = df.loc[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec', na=False), 'body']

In [8]:
inflation.dropna(inplace=True)

In [9]:
inflation = inflation.reset_index()

In [10]:
inflation.insert(loc=1, column='topics', value='inflation')

In [11]:
inflation.head(10)

,index,topics,body
0,1,inflation,Argentina sirve como uno de los países más par...
1,8,inflation,"El economista Steve Hanke, profesor e investig..."
2,20,inflation,El INDEC divulgó las cifras oficiales para el ...
3,22,inflation,Desde la toma ilegal del poder de Maduro hace ...
4,24,inflation,En un año marcado por las nuevas regulaciones ...
5,31,inflation,"Esta mañana, el ministro Guzmán lanzó un inusu..."
6,41,inflation,En una reunión virtual de la Fundación Mediter...
7,43,inflation,Tras conocerse la estimación del Gobierno en t...
8,54,inflation,Según la ultima actualización realizada por el...
9,56,inflation,"El Índice de Precios al Consumidor, anunciado ..."


In [12]:
exchange = df.loc[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo', na=False), 'body']

In [13]:
exchange.dropna(inplace=True)

In [14]:
exchange = exchange.reset_index()

In [15]:
exchange.insert(loc=1, column='topics', value='exchange')

In [16]:
exchange.head(10)

,index,topics,body
0,2,exchange,"Mark Mobius, el reconocido gestor de mercados ..."
1,8,exchange,"El economista Steve Hanke, profesor e investig..."
2,13,exchange,"Según un reciente informe de Bloomberg, fuente..."
3,18,exchange,Hoy el dólar libre llegó a venderse a $174 en ...
4,39,exchange,Después de casi dos semanas de constantes aume...
5,42,exchange,El día de hoy finaliza una semana signada por ...
6,49,exchange,La progresiva radicalización hacia la izquierd...
7,55,exchange,El 15 de septiembre pasado el Banco Central an...
8,60,exchange,El caos cambiario volvió rápidamente a la esce...
9,63,exchange,El acuerdo con los acreedores internacionales ...


In [17]:
corpus = inflation.append(exchange)
corpus.shape

(67, 3)

In [18]:
corpus.sample(10)

,index,topics,body
4,24,inflation,En un año marcado por las nuevas regulaciones ...
26,215,inflation,"El índice IPC-OJF, elaborado por el Centro de ..."
15,102,inflation,"Con las ligeras medidas de flexibilización, pr..."
19,166,exchange,En un golpe de confianza producido por la recu...
18,132,inflation,"Se estimaba que para junio el Banco Central, d..."
7,55,exchange,El 15 de septiembre pasado el Banco Central an...
27,248,exchange,La jornada de hoy se caracterizó por una fuert...
12,74,exchange,La semana se caracterizó por una sucesión de d...
9,56,inflation,"El Índice de Precios al Consumidor, anunciado ..."
20,184,inflation,El Ministerio de Economía relevó ayer el últim...


I drop those news labeled under both topics

In [19]:
corpus = corpus.drop_duplicates(subset='index',keep=False)

In [20]:
corpus.shape

(61, 3)

### Tokenization
#### Manual

In [21]:
import re
import unidecode

"Stop words", special characters, accents and numbers are removed

In [22]:
stopwords_sp = stopwords.words('spanish')

In [23]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [24]:
tokenized = [tokenizer(documento) for documento in corpus.body]

In [25]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [26]:
len(vocabulario)

4699

#### CountVectorizer

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode')

In [29]:
cleaning = [re.sub(r'(\d|\$|\%|\+)', '', doc) for doc in corpus.body]

In [30]:
vectorizer.fit(cleaning)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian'

CountVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [31]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(4521, 4521)

I prefare the manual tokenization results since the process is clearer and there are more terms.

### Prepare tokens and pickle

In [32]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [33]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [34]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [35]:
# add classification column
topics = ['inflation' for i in range((corpus.topics == 'inflation').sum())] + ['exchange' for i in range((corpus.topics == 'exchange').sum())]
token_df.insert(loc=0, column='topics', value=topics)

In [36]:
token_df

,topics,abajo,abandonara,abandone,abastecimiento,abatida,abierta,abierto,aboga,abonados,...,woyecheszen,xx,yaguarete,york,ypf,yuanes,zamora,zona,zonaprop,zonas
doc1,inflation,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
doc2,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
doc3,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,0
doc5,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc57,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc58,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc59,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc60,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
token_df.to_pickle('tokens/topic_tokens_der.pkl')